In [1]:
from dataInput import *
from CV import *
from TwoStageSP import *
from FA import *
import numpy as np
import pandas as pd

In [2]:
inputParams = inputParams(1,1,57,1000);
solveParams = solveParams(100000,200,100000,10800,1e-4);

In [3]:
intensityFile = 'data/synthetic/intensity.csv';
locationFile = 'data/synthetic/location.csv';
landfallFile = 'data/synthetic/landfall_7.csv';

In [4]:
hurricaneDataSet = hurricaneInput(intensityFile, locationFile, landfallFile, inputParams)

In [5]:
netNodesFile = 'data/synthetic/nodes.csv';
netParamsFile = 'data/synthetic/netParams.csv';
networkDataSet = networkInput(3,10,0.05,netNodesFile,netParamsFile,hurricaneDataSet)

In [6]:
osfname = "./data/synthetic/OOS" + str(inputParams.k_init) + ".csv"
OS_paths = pd.read_csv(osfname).values  # Read the out-of-sample file

In [7]:
t_roll = 0;
x_init = networkDataSet.x_0;
k_t = inputParams.k_init-1

In [8]:
master, x, f, theta, subproblem, y2, xCons, dCons, rCons = RH_2SSP_define_models(networkDataSet, hurricaneDataSet, inputParams, t_roll, k_t, x_init)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-04


In [ ]:
Ni = networkDataSet.Ni;
Nj = networkDataSet.Nj;
N0 = networkDataSet.N0;
T = hurricaneDataSet.T;
absorbing_option = inputParams.absorbing_option;
nodeScenList = hurricaneDataSet.nodeScenList;
nodeScenWeights = hurricaneDataSet.nodeScenWeights;

nbstages1 = T - t_roll
if absorbing_option == 0:
        # If no MDC/SP operation is allowed in the absorbing state, do not plan for stage T since we know for sure that all states are absorbing
        nbstages1 = T - t_roll - 1
nbScens = len(nodeScenList[(t_roll,k_t)])
pbScens = nodeScenWeights[(t_roll,k_t)]    
LB = -1e10
UB = 1e10
thetaval = [0] * nbScens
xval = [[0] * nbstages1 for _ in range(Ni)]
fval = [[[0] * nbstages1 for _ in range(Ni)] for _ in range(N0)]
solveIter = 0
flag = 1

In [ ]:
LB, xval, fval, thetaval = solve_first_stage(networkDataSet, master, x, f, theta, nbstages1, nbScens)
firstCost = LB - sum(thetaval[n] * pbScens[n] for n in range(nbScens))
flag, Qbar = solve_second_stage(networkDataSet, hurricaneDataSet, inputParams, solveParams, k_t, t_roll, xval, fval, thetaval, master, subproblem, x, f, theta, y2, xCons, dCons, rCons)
if flag != -1:
    UB = min(firstCost + Qbar, UB)

In [ ]:
LB

In [ ]:
UB

In [ ]:
xval

In [ ]:
cb = networkDataSet.cb;
ch = networkDataSet.ch;
cp = networkDataSet.cp;

n = 117;
absorbingT = nodeScenList[(t_roll,k_t)][n][0]
absorbingState = nodeScenList[(t_roll,k_t)][n][1]
RH_2SSP_update_RHS(networkDataSet, hurricaneDataSet, inputParams, absorbingT, absorbingState, xCons, dCons, rCons, xval, fval, y2, t_roll)
Q, pi1, pi2, pi3, flag = solve_scen_subproblem(networkDataSet, subproblem, xCons, dCons, rCons)
print("rCons = %f" % rCons.getAttr(GRB.Attr.RHS))
print("absorbingT = %d, absorbingState = %d" %(absorbingT, absorbingState))
print("Q = %f\n" % Q)
for i in range(Ni):
    for j in range(Nj):
        print(y2[i,j].Obj);
        
print("ca = ");        
ca = networkDataSet.ca;
for i in range(Ni):
    for j in range(Nj):
        print(ca[i,j,2]);

In [ ]:
RH_2SSP_solve_roll(networkDataSet, hurricaneDataSet, inputParams, solveParams, k_t, t_roll, master, subproblem, x, f, theta, y2, xCons, dCons, rCons)

In [9]:
RH_eval(networkDataSet, hurricaneDataSet, inputParams, solveParams, osfname)

training LB = 6518.262780191745
training UB = 6513.130576208083
static 2SSP....
μ ± 1.96*σ/√NS = 6018.141648677899 ± 772.8290539208651


[[4137.994473777085,
  4240.913921889643,
  4118.473752613034,
  3873.5934140598156,
  4430.9805703194,
  4137.994473777085,
  4187.778065340189,
  4303.712566704546,
  4430.9805703194,
  4341.601931609268,
  3751.1532447832064,
  4430.9805703194,
  4303.712566704546,
  4430.9805703194,
  4118.473752613034,
  4434.72388324516,
  4303.712566704546,
  4064.2055327546564,
  3873.5934140598156,
  3751.1532447832064,
  4176.444563089691,
  3873.5934140598156,
  3751.1532447832064,
  3873.5934140598156,
  3996.0335833364247,
  4187.778065340189,
  3873.5934140598156,
  4395.691003521655,
  3873.5934140598156,
  3996.0335833364247,
  4434.72388324516,
  4015.23135203713,
  4167.987863472282,
  3873.5934140598156,
  4386.539407193588,
  4303.712566704546,
  3751.1532447832064,
  4687.985717686584,
  3751.1532447832064,
  4118.473752613034,
  3873.5934140598156,
  3751.1532447832064,
  4303.712566704546,
  4434.72388324516,
  4874.8440062143545,
  4363.354091166251,
  4303.712566704546,
  4167.